# DADA2 ASV pipeline

This notebook uses the [DADA2](https://benjjneb.github.io/dada2/index.html) pipeline to infer ASVs from 16S rhizosphere samples.

## Setup

The analysis was performed using dada2 1.10.0 with R 3.5.1. The computational environment used to carry out the analysis can be replicated on a Linux system using [conda](https://docs.conda.io/en/latest/miniconda.html). Download the miniconda installer [here](https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh) then follow the [installation instruction](https://docs.conda.io/en/latest/miniconda.html#installing).

Once conda is available on your system, the dada2 environment can be created by running the following command from within the directory containing this notebook:

```
conda env create -f dada2_1.10.0.conda.yaml
```

The conda environment can then be activate using 

```
conda activate dada2_1.10.0
```

## Data Preparation

Demultiplexed fastq files for all samples should be downloaded into the `fastq` directory:

<font color="red">TODO: Add script to download from ENA once demultiplexed samples available</font>

Download the silva v138 database from [zenodo: doi:10.5281/zenodo.3731176](https://zenodo.org/record/3731176#.YVMc-HnTVTY)

The following code cell will download the appropriate file and confirm the integrity of the downloaded file.

In [26]:
import urllib.request
import hashlib
import sys
import os

dl_uri='https://zenodo.org/record/3731176/files/silva_nr_v138_train_set.fa.gz?download=1'
fn='silva_nr_v138_train_set.fa.gz'
good_md5sum='1deeaa2ecc9dbeabdcb9331a565f8343'

def file_read(file):
    with file:
        return file.read()

urllib.request.urlretrieve(dl_uri,fn)
dl_md5sum=hashlib.md5(file_read(open(fn,'rb'))).hexdigest()

if good_md5sum != dl_md5sum:
    sys.stdout.write('Downloaded file appears corrupted')
    os.remove(fn)

## Running the DADA2 analysis

The scripts to run this analysis are optimised for use on an HPC cluster running Univa Grid Engine/Sun Grid Engine. It will probably be necessary to modify the GridEngine directives at the top of each script to make them suitable for your compute environment. The scripts are designed for use directly from the command line and can be run interactively, but would ideally be submitted to an HPC system.

The process is split into three parts for optimal HPC cluster usage. 

  *  *dada2_00.R*: QC and read trimming. Runs multithreaded for improved runtimes, using 24 threads by default. To adjust the number of threads change the value of 'num_threads' at the top of the script, and if submitting to an HPC environment then the value of the GridEngine `pe` directive should be altered to match the value of num_threads. Usage information for the script is as follows:
  
```
Usage: ./dada2_00.R [-[-reads|r] <character>] [-[-metadata|m] <character>] [-[-truncF|F] [<integer>]] [-[-truncR|R] [<integer>]] [-[-maxeeF|q] [<integer>]] [-[-maxeeR|w] [<integer>]] [-[-help|h]]
    -r|--reads       Path to directory of fastq files
    -m|--metadata    Path to metadata file
    -F|--truncF      Length to truncate forward reads to (default: no truncation)
    -R|--truncR      Length to truncate forward reads to (default: no truncation)
    -q|--maxeeF      Maximum expected errors for forward reads (default: 2)
    -w|--maxeeR      Maximum expected errors for reverse reads (default: 2)
    -h|--help        Display help
 ```
 
 The necessary metadata file is provided in this directory named `Map_JH12_2.txt`. Interim results are stored as serialiased R objects in a `cache` directory.  
 
For this analysis, the script was run by submission to the HPC cluster as follows:

In [ ]:
%%bash
qsub dada2_00.R --truncF 150 --truncR 150 --reads fastq --metadata Map_JH12_2.txt

  * *dada2_01.R*: learns error rates, dereplicates sequences and carries out pooled ASV inference using dada2().
  
    
Submitting this script to a GridEngine cluster will run an array job consisting of two tasks, one each for the forward and reverse read which are processed separately for effeciency. dada2 is run for a maximum of 20 rounds or until convergence with the error model. Again, this script is multithreaded, and the number of threads can be adjusted as described above for `dada2_00.R`. Usage of the script is as follows:
    
    
```
Usage: ./dada2_01.R [-[-metadata|m] <character>] [-[-help|h]]
    -m|--metadata    Path to metadata file
    -h|--help        Display help
```
    
In this case, only the metadata file needs to be provided to the script, with results from `dada2_00.R` automatically reloaded from the `cache` directory.

The script was run as follows:

In [ ]:
%%bash
qsub dada2_01.R --metadata Map_JH12_2.txt

  * *dada2_02.R*: Merges paired reads, removes chimeras and carrries out ASV classification. A phyloseq object is created containing the resulting classified ASVs and serialised into the working directory as `JH12_dada2.rds`. 
  
  As with previous scripts, the number of threads can be adjusted by altering `num_threads` at the top of the script. Usage is as follows:
  
  
```
Usage: ./dada2_02.R [-[-name|n] <character>] [-[-taxonomy|t] <character>] [-[-merge_overlap|m] [<character>]] [-[-help|h]]
    -n|--name             Name of job
    -t|--taxonomy         Path to taxonomy database to use for classification
    -m|--merge_overlap    Minimum overlap for merging reads (default: 12)
    -h|--help             Display help
```
 
The script was run as follows:

In [ ]:
%%bash
qsub dada2_02.R -n JH12 -t silva_nr_v138_train_set.fa.gz